In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Input, Bidirectional,SimpleRNN, Embedding, Conv1D, LSTM, MaxPooling1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split



from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import seaborn as sns

print(tf.__version__)


2024-08-26 11:28:45.887844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hsibitenda/anaconda3/envs/harriet_env/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
[nltk_data] Downloading package stopwords t

2.12.0


In [2]:
# !pip install pyecharts

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
import torch

print(torch.cuda.is_available())

if torch.cuda.is_available():
    print("Cuda is Availabe")
else:
    print("Cuda Can't be found")

True
Cuda is Availabe


In [63]:
# df = pd.read_csv('D:/senego/dataafrica/socialcons_clean2.csv')
# df = pd.read_csv('D:/senego/dataafrica/socialcons_clean2.csv')
#df = pd.read_csv('C:/Users/Administrator/OneDrive/Documents/africa/dataafrica/socialcons_clean.csv', encoding = "UTF-8")
# df=pd.read_csv('D:/senego/dataafrica/analysis/bert.csv')
# df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/new_dataset2_aspects.csv')
df=pd.read_csv('/home/hsibitenda/scratch/harriet/sentiment_sentall_llama32new.csv')

df=df.astype(str)
df.head(5)

,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,Sentiment2_prediction,Sentiment2_roberta,Predicted_Labels_roberta,Predicted_Sentiment_finetunned,mapped_initial_label,finetunned_BERT,finetunned_RoBERTa,finetunned_Llama3,BERT_nlptown,cleaned_theme
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,neutral,neutral,1,neutral,poverty,neutral,neutral,neutral,neutral,poverty
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,negative,negative,0,negative,poverty,negative,negative,negative,negative,poverty
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,negative,negative,0,negative,poverty,negative,negative,negative,negative,poverty
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,positive,positive,2,positive,poverty,positive,positive,positive,positive,poverty
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,negative,negative,0,negative,poverty,negative,negative,negative,negative,poverty


In [5]:
df.shape

(22036, 62)

In [64]:
df3=df.copy()
df4=df.copy()
df=df3.head(50)
df.shape

(50, 62)

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np, os 
import pandas as pd, gc 
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForTokenClassification,LukeForEntitySpanClassification,RobertaTokenizer,RobertaTokenizerFast,LukeTokenizer,DataCollatorForTokenClassification,TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset as ds, DatasetDict, load_metric

from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import accuracy_score
from torch import cuda
from torch import nn
import itertools

# !pip install seqeval
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
# import torch
# config = {
#     'model_name': 'studio-ousia/luke-large-finetuned-conll-2003',
#     'model_path': "C:/Users/Administrator/transformers/src/transformers/models/luke/",
#     'max_length': 512,
#     'train_batch_size': 2,
#     'valid_batch_size': 4,
#     'epochs': 1,
#     'learning_rates': [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7],
#     'max_grad_norm': 10,
#     'device': 'cuda' if torch.cuda.is_available() else 'cpu'
# }


In [9]:
# Currently, LUKE is only available on the master branch
# !pip install seqeval git+https://github.com/huggingface/transformers.git

In [10]:
import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification

In [11]:
# pip install wget
# Currently, LUKE is only available on the master branch
# !pip install seqeval git+https://github.com/huggingface/transformers.git

In [11]:
# Download the testb set of the CoNLL-2003 dataset
!curl https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

-DOCSTART- -X- -X- O

SOCCER NN I-NP O
- : O O
JAPAN NNP I-NP I-LOC
GET VB I-VP O
LUCKY NNP I-NP O
WIN NNP I-NP O
, , O O
CHINA NNP I-NP I-PER
IN IN I-PP O
SURPRISE DT I-NP O
DEFEAT NN I-NP O
. . O O

Nadim NNP I-NP I-PER
Ladki NNP I-NP I-PER

AL-AIN NNP I-NP I-LOC
, , O O
United NNP I-NP I-LOC
Arab NNP I-NP I-LOC
Emirates NNPS I-NP I-LOC
1996-12-06 CD I-NP O

Japan NNP I-NP I-LOC
began VBD I-VP O
the DT I-NP O
defence NN I-NP O
of IN I-PP O
their PRP$ I-NP O
Asian JJ I-NP I-MISC
Cup NNP I-NP I-MISC
title NN I-NP O
with IN I-PP O
a DT I-NP O
lucky JJ I-NP O
2-1 CD I-NP O
win VBP I-VP O
against IN I-PP O
Syria NNP I-NP I-LOC
in IN I-PP O
a DT I-NP O
Group NNP I-NP O
C NNP I-NP O
championship NN I-NP O
match NN I-NP O
on IN I-PP O
Friday NNP I-NP O
. . O O

But CC O O
China NNP I-NP I-LOC
saw VBD I-VP O
their PRP$ I-NP O
luck NN I-NP O
desert VB I-VP O
them PRP I-NP O
in IN I-PP O
the DT I-NP O
second NN I-NP O
match NN I-NP O
of IN I-PP O
the DT I-NP O
group NN I-NP O
, , O O
crashing V

In [12]:
# def load_examples(dataset_file):
#     with open(dataset_file, "r") as f:
#         data = json.load(f)

#     examples = []
#     for item in data:
#         examples.append(dict(
#             text=item["sent"],
#             entity_spans=[(item["start"], item["end"])],
#             label=item["labels"]
#         ))

#     return examples

In [27]:
# !gdown --id 1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im

In [25]:
import json
import torch
from tqdm import trange
from transformers import LukeTokenizer, LukeForEntityClassification
!gdown --id 1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
# test_examples = load_examples("test.json")

# Download the testb set of the CoNLL-2003 dataset
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

/bin/bash: gdown: command not found
--2024-08-26 11:43:50--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb’

eng.testb           100%[===================>] 730.56K  --.-KB/s    in 0.05s   

2024-08-26 11:43:50 (15.0 MB/s) - ‘eng.testb’ saved [748096/748096]



In [14]:
# # Extract the tar.gz file using the absolute path
# !tar -xzf "C:/Users/Administrator/OneDrive/Documents/africa/entities/data.tar.gz"

# # Copy test.json to the current directory using xcopy
# !xcopy "C:/Users/Administrator/OneDrive/Documents/africa/entities/data/OpenEntity/test.json" .


In [15]:
import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification
# model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003", from_tf=True)


In [17]:
from torch import cuda
import torch
from transformers import LukeForEntitySpanClassification, LukeTokenizer

# path_to_pytorch_bin_file = "C:/Users/Administrator/.cache/huggingface/hub/models--studio-ousia--luke-large-finetuned-conll-2003/snapshots/23dbd7b4e1d28e07470f63d79cd206fe15a23b25/pytorch_model.bin"

# try:
#     # Attempt to load the model checkpoint
#     state_dict = torch.load(path_to_pytorch_bin_file, map_location="cpu")
# except Exception as e:
#     print(f"Error loading the model checkpoint: {e}")
#     raise

# The rest of your code...


model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda" if cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))

    return documents


def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end + 1]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [21]:
# df=df3.head(50)

In [22]:
# df.head()

In [26]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

# test_documents = df
# test_examples = load_examples(test_documents)


100%|██████████| 231/231 [00:00<00:00, 274.70it/s]


In [28]:
batch_size = 2
all_logits = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda" if cuda.is_available() else "cpu")
    with torch.no_grad():
        outputs = model(**inputs)
    all_logits.extend(outputs.logits.tolist())

100%|██████████| 1727/1727 [01:12<00:00, 23.96it/s]


In [29]:
final_labels = [label for document in test_documents for label in document["labels"]]

final_predictions = []
for example_index, example in enumerate(test_examples):
    logits = all_logits[example_index]
    max_logits = np.max(logits, axis=1)
    max_indices = np.argmax(logits, axis=1)
    original_spans = example["original_word_spans"]
    predictions = []
    for logit, index, span in zip(max_logits, max_indices, original_spans):
        if index != 0:  # the span is not NIL
            predictions.append((logit, span, model.config.id2label[index]))

    # construct an IOB2 label sequence
    predicted_sequence = ["O"] * len(example["words"])
    for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
            predicted_sequence[span[0]] = "B-" + label
            if span[1] - span[0] > 1:
                predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

    final_predictions += predicted_sequence

In [30]:

print(seqeval.metrics.classification_report([final_labels], [final_predictions], digits=4))

              precision    recall  f1-score   support

         LOC     0.9558    0.9478    0.9518      1666
        MISC     0.8553    0.8688    0.8620       701
         ORG     0.9287    0.9496    0.9391      1647
         PER     0.9683    0.9719    0.9701      1602

   micro avg     0.9386    0.9453    0.9420      5616
   macro avg     0.9270    0.9345    0.9307      5616
weighted avg     0.9389    0.9453    0.9421      5616



In [70]:
# df

## Recognizing named entities in a text

Finally, we extract named entities from a text using the [fine-tuned model](https://huggingface.co/studio-ousia/luke-large-finetuned-conll-2003). The input text is tokenized using [SpaCy](https://spacy.io/).

In [31]:
text = "Star Wars is a film written and directed by George Lucas"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

entity_spans = []
original_word_spans = []
for token_start in doc:
    for token_end in doc[token_start.i:]:
        entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
        original_word_spans.append((token_start.i, token_end.i + 1))

inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt", padding=True)
inputs = inputs.to("cuda" if cuda.is_available() else "cpu")
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
max_logits, max_indices = logits[0].max(dim=1)

predictions = []
for logit, index, span in zip(max_logits, max_indices, original_word_spans):
    if index != 0:  # the span is not NIL
        predictions.append((logit, span, model.config.id2label[int(index)]))

# construct an IOB2 label sequence
predicted_sequence = ["O"] * len(doc)
for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
    if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
        predicted_sequence[span[0]] = "B-" + label
        if span[1] - span[0] > 1:
            predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

for token, label in zip(doc, predicted_sequence):
    print(token, label)

Star B-MISC
Wars I-MISC
is O
a O
film O
written O
and O
directed O
by O
George B-PER
Lucas I-PER


In [32]:
text = "Star Wars is a film written and directed by George Williams in Senegal"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

entity_spans = []
original_word_spans = []
for token_start in doc:
    for token_end in doc[token_start.i:]:
        entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
        original_word_spans.append((token_start.i, token_end.i + 1))

inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt", padding=True)
inputs = inputs.to("cuda")
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
max_logits, max_indices = logits[0].max(dim=1)

predictions = []
for logit, index, span in zip(max_logits, max_indices, original_word_spans):
    if index != 0:  # the span is not NIL
        predictions.append((logit, span, model.config.id2label[int(index)]))

# construct an IOB2 label sequence
predicted_sequence = ["O"] * len(doc)
for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
    if all([o == "O" for o in predicted_sequence[span[0] : span[1]]]):
        predicted_sequence[span[0]] = "B-" + label
        if span[1] - span[0] > 1:
            predicted_sequence[span[0] + 1 : span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

for token, label in zip(doc, predicted_sequence):
    print(token, label)

Star B-MISC
Wars I-MISC
is O
a O
film O
written O
and O
directed O
by O
George B-PER
Williams I-PER
in O
Senegal B-LOC


In [33]:
# df=df3.copy()
df=df3.head(50)
df.shape

(50, 62)

In [35]:
import torch
import pandas as pd
from transformers import LukeTokenizer, LukeForEntitySpanClassification
import spacy

# Load the LUKE model and tokenizer
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

# Define a function to process each chunk of text
def process_chunk(chunk_text):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(chunk_text)
    return doc

# Define a function to extract entities, original word spans, predictions, and tokens with sequence labels
def extract_info(row):
    max_sequence_length = 128
    text = row['Text_lemma'][:max_sequence_length]
    chunk_size = 5000

    for i in range(0, len(text), chunk_size):
        chunk_text = text[i:i + chunk_size]
        doc = process_chunk(chunk_text)  # Call the process_chunk function

        entity_spans = []
        original_word_spans = []
        entities = set()  # Use a set to store unique entities

        # Process text in chunks of 64 tokens
        for chunk_start in range(0, len(doc), 128):
            chunk_end = min(chunk_start + 128, len(doc))
            chunk = doc[chunk_start:chunk_end]

            for token_start in chunk:
                for token_end in doc[token_start.i:]:
                    entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
                    original_word_spans.append((token_start.i, token_end.i + 1))
                    entities.add(chunk_text[token_start.idx:token_end.idx + len(token_end)])  # Extract the entity text and add to the set

        inputs = tokenizer(chunk_text, entity_spans=entity_spans, max_length=128, padding='max_length', return_tensors="pt")
        inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        max_logits, max_indices = logits[0].max(dim=1)

        predictions = []

        for logit, index, span in zip(max_logits, max_indices, original_word_spans):
            if index != 0:  # the span is not NIL
                predictions.append((logit.item(), span, model.config.id2label.get(int(index), 'UNKNOWN_LABEL')))

        # construct an IOB2 label sequence
        predicted_sequence = ["O"] * len(doc)
        for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
            if all([o == "O" for o in predicted_sequence[span[0]:span[1]]]):
                predicted_sequence[span[0]] = "B-" + label
                if span[1] - span[0] > 1:
                    predicted_sequence[span[0] + 1:span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

        tokens_with_labels = [f"{token.text} ({label})" for token, label in zip(doc, predicted_sequence)]

        # Extract text within the spans identified by the predicted sequence with non-'O' labels
        text_in_spans = [token.text for token, label in zip(doc, predicted_sequence) if label != "O"]
        row['text_in_spans'] = " ".join(text_in_spans)

        # Extract entity pairs with labels other than 'O'
        relevant_entity_spans = [(start, end, label) for (start, end), label in zip(entity_spans, predicted_sequence) if label != "O"]
        if relevant_entity_spans:
            # If spans have 'B' or 'I' labels
            entities_of_interest = ['LOC', 'ORG', 'PER']
            relevant_entities = [(start, end) for (start, end, label) in relevant_entity_spans if label[2:] in entities_of_interest]
            if len(relevant_entities) >= 2:
                # If there are at least two relevant entities, return the first two pairs
                row['entity_spans'] = relevant_entities[:2]
            elif len(relevant_entities) == 1:
                # If there is only one relevant entity, return the first and last pairs
                row['entity_spans'] = [relevant_entities[0], relevant_entities[0]]
            else:
                # If no relevant entities, return the first and last span pair
                row['entity_spans'] = [(entity_spans[0][0], entity_spans[-1][1]), (0, 0)]
        else:
            # If no spans with 'B' or 'I' labels, return the first and last span pair
            row['entity_spans'] = [(entity_spans[0][0], entity_spans[-1][1]), (0, 0)]

        # Add new columns 'categories', 'entity_spans', 'original_word_spans', 'predictions', 'tokens_with_labels'
        row['categories'] = [label for _, _, label in predictions]
        row['entity'] = entity_spans
        row['original_word_spans'] = original_word_spans
        row['predictions'] = predictions
        row['tokens_with_labels'] = tokens_with_labels

    return row

# Define the chunk size
chunk_size = 5000

# Process the DataFrame in chunks
dfs = []
for i in range(0, len(df), chunk_size):
    chunk_df = df.iloc[i:i + chunk_size]
    chunk_df = chunk_df.apply(extract_info, axis=1)
    chunk_df.to_csv(f'output_chunk_{i // chunk_size}.csv', index=False)  # Adjusted the numbering
    dfs.append(chunk_df)

# Merge the small DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Save the final DataFrame to a CSV file
final_df.to_csv('/home/hsibitenda/scratch/harriet/entities_code/output_luke_final_new3.csv', index=False)
# df=pd.read_csv('/home/hsibitenda/scratch/harriet/sentiment_sentall_llama32new.csv')

final_df.head()


Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,finetunned_Llama3,BERT_nlptown,cleaned_theme,text_in_spans,entity_spans,categories,entity,original_word_spans,predictions,tokens_with_labels
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,neutral,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",[LOC],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O..."
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,negative,negative,poverty,,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ..."
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,negative,negative,poverty,,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom..."
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,positive,positive,poverty,,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa..."
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,negative,negative,poverty,african,"[(0, 99), (0, 0)]",[MISC],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ..."


In [37]:
# final_df['entity_spans']

In [38]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import LukeTokenizer, LukeForEntityPairClassification

# Load the LUKE model and tokenizer
model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
max_seq_length = 128  # Adjust as needed

# Create a list to store predicted labels
predicted_labels = []

# Process each row in the DataFrame
for _, row in tqdm(final_df.iterrows(), total=len(final_df)):
    text = row['Text_lemma']
    entity_spans = row['entity_spans']

    # Ensure entity_spans is a list of two tuples
    if not isinstance(entity_spans, list) or len(entity_spans) != 2 or not all(isinstance(pair, tuple) and len(pair) == 2 for pair in entity_spans):
        # Handle the case where 'entity_spans' is not in the expected format
        start_entity_pairs = end_entity_pairs = [0, 0]
    else:
        # Extract the first and last pairs from the list
        start_entity_pairs = [entity_spans[0][0], entity_spans[-1][0]]
        end_entity_pairs = [entity_spans[0][1], entity_spans[-1][1]]

    # Tokenize and truncate the input sequence
    inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt", truncation=True, max_length=max_seq_length)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class_idx = outputs.logits.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_idx]

    # Append the predicted class to the list
    predicted_labels.append(predicted_class)

# Add the predicted labels to the DataFrame
final_df['predicted_labels'] = predicted_labels

# Display the DataFrame with the predicted labels
final_df.head()


config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-tacred were not used when initializing LukeForEntityPairClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 50/50 [00:00<00:00, 64.52it/s]


,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,BERT_nlptown,cleaned_theme,text_in_spans,entity_spans,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",[LOC],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O...",no_relation
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,negative,poverty,,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ...",no_relation
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,negative,poverty,,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom...",no_relation
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,positive,poverty,,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa...",no_relation
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,negative,poverty,african,"[(0, 99), (0, 0)]",[MISC],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ...",no_relation


In [7]:
# final_df

In [39]:
# final_df.to_csv('D:/senego/dataafrica/analysis/output_luke_final_new31.csv', index=False)
final_df.to_csv('../output_luke_final_new31.csv')#, index=False)


In [40]:
final_df.shape

(50, 70)

In [41]:
df3=final_df.copy()

In [42]:
df=final_df.head(50)
# df=df.astype(str)
df.shape

(50, 70)

In [43]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0', 'textid', 'Date', 'Views', 'sourcetype',
       'alltext', 'Text_emt', 'Text_emj', 'Text_langue', 'Text_translate',
       'Text_langue2', 'Text_lower', 'Text_punct', 'Text_tags', 'Text_chat',
       'Text_spell', 'Text_langue3', 'Text_translate2', 'Text_token',
       'Text_stop', 'Text_stem', 'Text_lemma', 'berttopic2', 'cluster_id2',
       'llama2_labelone10', 'cluster_labelone10', 'cluster_id',
       'llama2_labelone', 'cluster_labelone', 'Unnamed: 0.1', 'Text_lemma2',
       'Bertscore_Labels', 'Final_Labels', 'Predicted_Categories3', 'polarity',
       'subjectivity', 'Textblob', 'VADER', 'neg', 'neu', 'pos', 'compound',
       'BERT', 'Sentiment_Score', 'Sentiwordnet', 'llama3_sentiment',
       'Emotion', 'Emotional_Polarity', 'Common_Sentiment32', 'Sentiment2',
       'Predicted_Labels', 'Sentiment2_prediction', 'Sentiment2_roberta',
       'Predicted_Labels_roberta', 'Predicted_Sentiment_finetunned',
       'mapped_initial_label', 'finet

In [51]:
# import pandas as pd
# # Create a new column 'span_pair'
# df['spans'] = df['predictions'].apply(lambda x: [span[1] for span in eval(x)] if x and isinstance(x, (str, list)) and any(span[1] for span in eval(x)) else [(0, 0)])

# # Display the DataFrame with the new column
# df.head()

import pandas as pd

# Assuming df is your DataFrame and 'predictions' contains string representations of lists or tuples.

# Define a safe eval function
def eval(x):
    try:
        # Ensure x is a string and not empty
        if isinstance(x, str) and x.strip():
            return eval(x)
        else:
            return []
    except Exception as e:
        print(f"Failed to eval: {x} with error {e}")
        return []

# Create a new column 'spans'
df['spans'] = df['predictions'].apply(lambda x: [span[1] for span in eval(x) if span and len(span) > 1] if x else [(0, 0)])

# Display the DataFrame with the new column
print(df.head())



  Unnamed: 0.2 Unnamed: 0 textid                       Date Views sourcetype  \
0            0          0  18965                 2021-08-24   576    youtube   
1            1          1    532  2019-08-22 12:41:52+00:00     0    twitter   
2            2          2   2876  2020-06-05 08:01:33+00:00     0    twitter   
3            3          3   2533  2020-09-15 03:04:00+00:00    12    twitter   
4            4          4   6413  2021-08-13 16:05:04+00:00    24    twitter   

                                             alltext  \
0    geography g.12- unit 4 population of ethiopi...   
1  what a waste of public money .worry about the ...   
2  you can't truely fixed economic; social and in...   
3  this made my day!! \nyou know the web and soci...   
4  @mczuriel @news24 @itchybyte don't worry yours...   

                                            Text_emt  \
0    geography g.12- unit 4 population of ethiopi...   
1  what a waste of public money .worry about the ...   
2  you can't t

/tmp/ipykernel_806199/2431752130.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spans'] = df['predictions'].apply(lambda x: [span[1] for span in eval(x) if span and len(span) > 1] if x else [(0, 0)])


In [52]:
# df['span_pair']

In [54]:
# # Create a new column 'start_pair'
# # Create a new column 'start_pair' using eval
# df['start_pair'] = df['entity'].apply(lambda x: eval(x)[0] if x else (0, 0))
# df['end_pair'] = df['spans'].apply(lambda x: (x)[0] if x else (0, 0))

# # # Create 'start_pair' and 'end_pair' columns
# # df['start_pair'] = df['entity'].apply(lambda x: x[0] if x else (0, 0))
# # df['end_pair'] = df['spans'].apply(lambda x: x[0] if x else (0, 0))

# # Create 'span_pairs' column by combining 'start_pair' and 'end_pair'
# df['span_pairs'] = df.apply(lambda row: [row['start_pair'], row['end_pair']], axis=1)

# #
# df['span_pairs']

import pandas as pd

# Assuming df is your DataFrame and 'entity' and 'spans' columns contain string representations of lists

# Create a safe eval function to handle possible eval issues
def safe_eval(x):
    try:
        # Evaluates the string if it's not None or empty
        if x:
            return eval(x)
        else:
            return []
    except Exception as e:
        print(f"Failed to evaluate: {x} with error: {e}")
        return []

# Create a new column 'start_pair' using eval with added safety checks
df['start_pair'] = df['entity'].apply(lambda x: safe_eval(x)[0] if safe_eval(x) else (0, 0))

# Assuming 'spans' contains lists of tuples, directly extracting the first element if present
df['end_pair'] = df['spans'].apply(lambda x: x[0] if x and isinstance(x, list) and len(x) > 0 else (0, 0))

# Create 'span_pairs' column by combining 'start_pair' and 'end_pair'
df['span_pairs'] = df.apply(lambda row: [row['start_pair'], row['end_pair']], axis=1)

# Print the 'span_pairs' column to see the results
print(df['span_pairs'].head())


0    [(0, 0), (0, 0)]
1    [(0, 0), (0, 0)]
2    [(0, 0), (0, 0)]
3    [(0, 0), (0, 0)]
4    [(0, 0), (0, 0)]
Name: span_pairs, dtype: object


/tmp/ipykernel_806199/1117329786.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_pair'] = df['entity'].apply(lambda x: safe_eval(x)[0] if safe_eval(x) else (0, 0))
/tmp/ipykernel_806199/1117329786.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end_pair'] = df['spans'].apply(lambda x: x[0] if x and isinstance(x, list) and len(x) > 0 else (0, 0))
/tmp/ipykernel_806199/1117329786.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [55]:
df.shape

(50, 74)

In [56]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import LukeTokenizer, LukeForEntityPairClassification

# Load the LUKE model and tokenizer
model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
max_seq_length = 128  # Adjust as needed

# Create a list to store predicted labels
predicted_labels = []

# Process each row in the DataFrame
for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Text_lemma']
    entity_spans = row['span_pairs']

    # Ensure entity_spans is a list of two tuples
    if not isinstance(entity_spans, list) or len(entity_spans) != 2 or not all(isinstance(pair, tuple) and len(pair) == 2 for pair in entity_spans):
        # Handle the case where 'entity_spans' is not in the expected format
        start_entity_pairs = end_entity_pairs = [0, 0]
    else:
        # Extract the first and last pairs from the list
        start_entity_pairs = [entity_spans[0][0], entity_spans[-1][0]]
        end_entity_pairs = [entity_spans[0][1], entity_spans[-1][1]]

    # Tokenize and truncate the input sequence
    inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt", truncation=True, max_length=max_seq_length)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class_idx = outputs.logits.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_idx]

    # Append the predicted class to the list
    predicted_labels.append(predicted_class)

# Add the predicted labels to the DataFrame
df['predicted_labels'] = predicted_labels

# Display the DataFrame with the predicted labels
df.head()


Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-tacred were not used when initializing LukeForEntityPairClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 50/50 [00:00<00:00, 69.20it/s]
/tmp/ipykernel_806199/1914695693.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels,spans,start_pair,end_pair,span_pairs
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,[LOC],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O...",no_relation,[],"(0, 0)","(0, 0)","[(0, 0), (0, 0)]"
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]"
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]"
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]"
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,[MISC],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ...",no_relation,[],"(0, 0)","(0, 0)","[(0, 0), (0, 0)]"


In [57]:
import torch
import pandas as pd
from transformers import LukeTokenizer, LukeForEntitySpanClassification
import spacy

# Load the LUKE model and tokenizer
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

# Define a function to process each chunk of text
def process_chunk(chunk_text):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(chunk_text)
    return doc

# Define a function to extract entities, original word spans, predictions, and tokens with sequence labels
def extract_info(row):
    max_sequence_length = 128
    text = row['Text_lemma'][:max_sequence_length]
    chunk_size = 5000

    entity_spans_list = []

    for i in range(0, len(text), chunk_size):
        chunk_text = text[i:i + chunk_size]
        doc = process_chunk(chunk_text)  # Call the process_chunk function

        entity_spans = []
        original_word_spans = []
        entities = set()  # Use a set to store unique entities

        # Process text in chunks of 64 tokens
        for chunk_start in range(0, len(doc), 128):
            chunk_end = min(chunk_start + 128, len(doc))
            chunk = doc[chunk_start:chunk_end]

            for token_start in chunk:
                for token_end in doc[token_start.i:]:
                    entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
                    original_word_spans.append((token_start.i, token_end.i + 1))
                    entities.add(chunk_text[token_start.idx:token_end.idx + len(token_end)])  # Extract the entity text and add to the set

        inputs = tokenizer(chunk_text, entity_spans=entity_spans, max_length=128, padding='max_length', return_tensors="pt")
        inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        max_logits, max_indices = logits[0].max(dim=1)

        predictions = []

        for logit, index, span in zip(max_logits, max_indices, original_word_spans):
            if index != 0:  # the span is not NIL
                predictions.append((logit.item(), span, model.config.id2label.get(int(index), 'UNKNOWN_LABEL')))

        # construct an IOB2 label sequence
        predicted_sequence = ["O"] * len(doc)
        for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
            if all([o == "O" for o in predicted_sequence[span[0]:span[1]]]):
                predicted_sequence[span[0]] = "B-" + label
                if span[1] - span[0] > 1:
                    predicted_sequence[span[0] + 1:span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

        tokens_with_labels = [f"{token.text} ({label})" for token, label in zip(doc, predicted_sequence)]

        # Extract text within the spans identified by the predicted sequence with non-'O' labels
        text_in_spans = [token.text for token, label in zip(doc, predicted_sequence) if label != "O"]
        row['text_in_spans'] = " ".join(text_in_spans)

        # Extract entity pairs with labels other than 'O'
        relevant_entity_spans = [(start, end, label) for (start, end), label in zip(entity_spans, predicted_sequence) if label != "O"]
        if relevant_entity_spans:
            # If spans have 'B' or 'I' labels
            entities_of_interest = ['LOC', 'ORG', 'PER']
            relevant_entities = [(start, end) for (start, end, label) in relevant_entity_spans if label[2:] in entities_of_interest]
            if len(relevant_entities) >= 2:
                # If there are at least two relevant entities, return all pairs
                entity_spans_list.extend(relevant_entities)
            elif len(relevant_entities) == 1:
                # If there is only one relevant entity, return the first and last pairs
                entity_spans_list.extend([relevant_entities[0], relevant_entities[0]])
            else:
                # If no relevant entities, return the first and last span pair
                entity_spans_list.extend([(entity_spans[0][0], entity_spans[-1][1]), (0, 0)])
        else:
            # If no spans with 'B' or 'I' labels, return the first and last span pair
            entity_spans_list.extend([(entity_spans[0][0], entity_spans[-1][1]), (0, 0)])

    # Add new columns 'categories', 'entity_spans', 'original_word_spans', 'predictions', 'tokens_with_labels'
    row['categories'] = [label for _, _, label in predictions]
    row['entity'] = entity_spans
    row['original_word_spans'] = original_word_spans
    row['predictions'] = predictions
    row['tokens_with_labels'] = tokens_with_labels

    # Split the 'entity_spans_list' into lists of two tuples
    entity_spans_list = [entity_spans_list[i:i+2] for i in range(0, len(entity_spans_list), 2)]
    entity_spans_list = [item for sublist in entity_spans_list for item in sublist]  # Flatten the list
    row['entity_spans_list'] = entity_spans_list

    return row


# Define the chunk size
chunk_size = 5000

# Process the DataFrame in chunks
dfs = []
for i in range(0, len(df), chunk_size):
    chunk_df = df.iloc[i:i + chunk_size]
    chunk_df = chunk_df.apply(extract_info, axis=1)
    chunk_df.to_csv(f'output_chunk_{i // chunk_size}.csv', index=False)  # Adjusted the numbering
    dfs.append(chunk_df)

# Merge the small DataFrames into one
df = pd.concat(dfs, ignore_index=True)
df.head()

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels,spans,start_pair,end_pair,span_pairs,entity_spans_list
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O...",no_relation,[],"(0, 0)","(0, 0)","[(0, 0), (0, 0)]","[(0, 36), (0, 36)]"
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]","[(0, 104), (0, 0)]"
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]","[(0, 128), (0, 0)]"
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa...",no_relation,"[(0, 0)]","(0, 0)","(0, 0)","[(0, 0), (0, 0)]","[(0, 127), (0, 0)]"
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ...",no_relation,[],"(0, 0)","(0, 0)","[(0, 0), (0, 0)]","[(0, 99), (0, 0)]"


In [58]:
df['predicted_labels']

0     no_relation
1     no_relation
2     no_relation
3     no_relation
4     no_relation
5     no_relation
6     no_relation
7     no_relation
8     no_relation
9     no_relation
10    no_relation
11    no_relation
12    no_relation
13    no_relation
14    no_relation
15    no_relation
16      per:title
17    no_relation
18    no_relation
19    no_relation
20    no_relation
21    no_relation
22    no_relation
23    no_relation
24    no_relation
25    no_relation
26    no_relation
27    no_relation
28    no_relation
29    no_relation
30    no_relation
31    no_relation
32    no_relation
33    no_relation
34    no_relation
35    no_relation
36    no_relation
37    no_relation
38    no_relation
39    no_relation
40    no_relation
41    no_relation
42    no_relation
43    no_relation
44    no_relation
45    no_relation
46    no_relation
47    no_relation
48    no_relation
49    no_relation
Name: predicted_labels, dtype: object

In [65]:
df=df4.copy()

In [66]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0', 'textid', 'Date', 'Views', 'sourcetype',
       'alltext', 'Text_emt', 'Text_emj', 'Text_langue', 'Text_translate',
       'Text_langue2', 'Text_lower', 'Text_punct', 'Text_tags', 'Text_chat',
       'Text_spell', 'Text_langue3', 'Text_translate2', 'Text_token',
       'Text_stop', 'Text_stem', 'Text_lemma', 'berttopic2', 'cluster_id2',
       'llama2_labelone10', 'cluster_labelone10', 'cluster_id',
       'llama2_labelone', 'cluster_labelone', 'Unnamed: 0.1', 'Text_lemma2',
       'Bertscore_Labels', 'Final_Labels', 'Predicted_Categories3', 'polarity',
       'subjectivity', 'Textblob', 'VADER', 'neg', 'neu', 'pos', 'compound',
       'BERT', 'Sentiment_Score', 'Sentiwordnet', 'llama3_sentiment',
       'Emotion', 'Emotional_Polarity', 'Common_Sentiment32', 'Sentiment2',
       'Predicted_Labels', 'Sentiment2_prediction', 'Sentiment2_roberta',
       'Predicted_Labels_roberta', 'Predicted_Sentiment_finetunned',
       'mapped_initial_label', 'finet

In [67]:
df.shape

(22036, 62)

In [70]:
import torch
import pandas as pd
from transformers import LukeTokenizer, LukeForEntitySpanClassification
import spacy

# Load the LUKE model and tokenizer
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

# Define a function to process each chunk of text
def process_chunk(chunk_text):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(chunk_text)
    return doc

# Define a function to extract entities, original word spans, predictions, and tokens with sequence labels
def extract_info(row):
    max_sequence_length = 128
    text = row['Text_lemma'][:max_sequence_length]
    chunk_size = 5000

    for i in range(0, len(text), chunk_size):
        chunk_text = text[i:i + chunk_size]
        doc = process_chunk(chunk_text)  # Call the process_chunk function

        entity_spans = []
        original_word_spans = []
        entities = set()  # Use a set to store unique entities

        # Process text in chunks of 64 tokens
        for chunk_start in range(0, len(doc), 128):
            chunk_end = min(chunk_start + 128, len(doc))
            chunk = doc[chunk_start:chunk_end]

            for token_start in chunk:
                for token_end in doc[token_start.i:]:
                    entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
                    original_word_spans.append((token_start.i, token_end.i + 1))
                    entities.add(chunk_text[token_start.idx:token_end.idx + len(token_end)])  # Extract the entity text and add to the set

        inputs = tokenizer(chunk_text, entity_spans=entity_spans, max_length=128, padding='max_length', return_tensors="pt")
        inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        max_logits, max_indices = logits[0].max(dim=1)

        predictions = []

        for logit, index, span in zip(max_logits, max_indices, original_word_spans):
            if index != 0:  # the span is not NIL
                predictions.append((logit.item(), span, model.config.id2label.get(int(index), 'UNKNOWN_LABEL')))

        # construct an IOB2 label sequence
        predicted_sequence = ["O"] * len(doc)
        for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
            if all([o == "O" for o in predicted_sequence[span[0]:span[1]]]):
                predicted_sequence[span[0]] = "B-" + label
                if span[1] - span[0] > 1:
                    predicted_sequence[span[0] + 1:span[1]] = ["I-" + label] * (span[1] - span[0] - 1)

        tokens_with_labels = [f"{token.text} ({label})" for token, label in zip(doc, predicted_sequence)]

        # Extract text within the spans identified by the predicted sequence with non-'O' labels
        text_in_spans = [token.text for token, label in zip(doc, predicted_sequence) if label != "O"]
        row['text_in_spans'] = " ".join(text_in_spans)

        # Extract entity pairs with labels other than 'O'
        relevant_entity_spans = [(start, end, label) for (start, end), label in zip(entity_spans, predicted_sequence) if label != "O"]
        if relevant_entity_spans:
            # If spans have 'B' or 'I' labels
            start_span = (relevant_entity_spans[0][0], relevant_entity_spans[0][1])
            end_span = (relevant_entity_spans[-1][0], relevant_entity_spans[-1][1])
            row['entity_pairs'] = [(start_span[0], start_span[1]), (end_span[0], end_span[1])]
        else:
            # If no spans with 'B' or 'I' labels, return the first and last span pair
            row['entity_pairs'] = [(entity_spans[0][0], entity_spans[-1][1]), (0, 0)]

        # Add new columns 'categories', 'entity_spans', 'original_word_spans', 'predictions', 'tokens_with_labels'
        row['categories'] = [label for _, _, label in predictions]
        row['entity'] = entity_spans
        row['original_word_spans'] = original_word_spans
        row['predictions'] = predictions
        row['tokens_with_labels'] = tokens_with_labels

    return row

# Define the chunk size
chunk_size = 5000

# Process the DataFrame in chunks
dfs = []
for i in range(0, len(df), chunk_size):
    chunk_df = df.iloc[i:i + chunk_size]
    chunk_df = chunk_df.apply(extract_info, axis=1)
    chunk_df.to_csv(f'output_chunk_{i // chunk_size}.csv', index=False)  # Adjusted the numbering
    dfs.append(chunk_df)

# Merge the small DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Save the final DataFrame to a CSV file
final_df.to_csv('../output_luke_final_new.csv', index=False)
final_df.head()


Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,finetunned_Llama3,BERT_nlptown,cleaned_theme,text_in_spans,entity_pairs,categories,entity,original_word_spans,predictions,tokens_with_labels
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,neutral,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",[LOC],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O..."
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,negative,negative,poverty,,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ..."
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,negative,negative,poverty,,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom..."
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,positive,positive,poverty,,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa..."
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,negative,negative,poverty,african,"[(0, 18), (0, 18)]",[MISC],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ..."


In [ ]:
# final_df

In [ ]:
# # final_df.shape
# final_df['entity_pairs'].head(10)

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import LukeTokenizer, LukeForEntityPairClassification

# Load the LUKE model and tokenizer
model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
max_seq_length = 128  # Adjust as needed

# Create a list to store predicted labels
predicted_labels = []

# Process each row in the DataFrame
for _, row in tqdm(final_df.iterrows(), total=len(final_df)):
    text = row['Text_lemma']
    entity_spans = row['entity_pairs']

    # Ensure entity_spans is a list of two tuples
    if not isinstance(entity_spans, list) or len(entity_spans) != 2 or not all(isinstance(pair, tuple) and len(pair) == 2 for pair in entity_spans):
        # Handle the case where 'entity_spans' is not in the expected format
        start_entity_pairs = end_entity_pairs = [0, 0]
    else:
        # Extract the first and last pairs from the list
        start_entity_pairs = [entity_spans[0][0], entity_spans[-1][0]]
        end_entity_pairs = [entity_spans[0][1], entity_spans[-1][1]]

    # Tokenize and truncate the input sequence
    inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt", truncation=True, max_length=max_seq_length)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class_idx = outputs.logits.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_idx]

    # Append the predicted class to the list
    predicted_labels.append(predicted_class)

# Add the predicted labels to the DataFrame
final_df['predicted_labels'] = predicted_labels

# Display the DataFrame with the predicted labels
final_df.head()


Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-tacred were not used when initializing LukeForEntityPairClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 22036/22036 [05:29<00:00, 66.78it/s]


,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,Text_langue,...,BERT_nlptown,cleaned_theme,text_in_spans,entity_pairs,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels
0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,en,...,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",[LOC],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), LOC)]","[geography (O), g (O), unit (O), population (O...",no_relation
1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,en,...,negative,poverty,,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[waste (O), public (O), money (O), worry (O), ...",no_relation
2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,en,...,negative,poverty,,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[ca (O), nt (O), truely (O), fixed (O), econom...",no_relation
3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,en,...,positive,poverty,,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"[made (O), know (O), web (O), medium (O), alwa...",no_relation
4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,en,...,negative,poverty,african,"[(0, 18), (0, 18)]",[MISC],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), MISC)]","[do (O), nt (O), worry (O), african (B-MISC), ...",no_relation


In [78]:
# final_df.to_csv('D:/senego/dataafrica/analysis/output_luke_final_new2.csv', index=False)
# final_df.to_csv('../output_luke_final_new2.csv')#, index=False)

# final_df=pd.read_csv('D:/senego/dataafrica/analysis/output_luke_final_new2.csv')#, index=False)
# final_df=final_df.astype(str)

In [90]:
# Optionally, save or display some of the predictions for manual inspection
final_df[['Text_lemma', 'predicted_labels']].head(10).to_csv('sample_predictions.csv')
print(final_df[['Text_lemma', 'predicted_labels']].head(10))

                                          Text_lemma predicted_labels
0  geography g unit population ethiopia horn afri...      no_relation
1  waste public money worry economy problem anc o...      no_relation
2  cant truely fixed economic infrastructural pro...      no_relation
3  made know web medium always trying start probl...      no_relation
4  dont worry african focus fixing useless countr...      no_relation
5  yeartrademarks pitched creative courageous ide...      no_relation
6  zana solving amp market challenge share impact...      no_relation
7  investing enterprise proven sustainably innova...      no_relation
8  make biggest change across young must encourag...      no_relation
9  need control medium number big problem fueling...      no_relation


In [136]:
final_df=pd.read_csv('../output_luke_final_new2.csv')#, index=False)
final_df.head(10)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,...,BERT_nlptown,cleaned_theme,text_in_spans,entity_pairs,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels
0,0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,...,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...",no_relation
1,1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,...,negative,poverty,NaN,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['waste (O)', 'public (O)', 'money (O)', 'worr...",no_relation
2,2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,...,negative,poverty,NaN,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['ca (O)', 'nt (O)', 'truely (O)', 'fixed (O)'...",no_relation
3,3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,...,positive,poverty,NaN,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['made (O)', 'know (O)', 'web (O)', 'medium (O...",no_relation
4,4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,...,negative,poverty,african,"[(0, 18), (0, 18)]",['MISC'],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), 'MISC')]","['do (O)', 'nt (O)', 'worry (O)', 'african (B-...",no_relation
5,5,5,5,1356,2019-08-09 16:40:01+00:00,42,twitter,at this yearâ€™s #baobab2019 #mcfscholars pitc...,at this yearâ€™s #baobab2019 #mcfscholars pitc...,at this yearâ€trade_marks #baobab2019 #mcfscho...,...,positive,poverty,yeartrademarks,"[(0, 14), (0, 14)]",['ORG'],"[(0, 14), (0, 22), (0, 31), (0, 42), (0, 47), ...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(1.5791925191879272, (0, 1), 'ORG')]","['yeartrademarks (B-ORG)', 'pitched (O)', 'cre...",no_relation
6,6,6,6,6008,2021-07-29 08:59:45+00:00,0,twitter,zana africa: solving social &amp; market chall...,zana africa: solving social &amp; market chall...,zana africa: solving social &amp; market chall...,...,positive,poverty,zana kenya,"[(0, 4), (0, 113)]","['ORG', 'LOC']","[(0, 4), (0, 12), (0, 16), (0, 23), (0, 33), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0.8383952379226685, (0, 1), 'ORG'), (2.63775...","['zana (B-ORG)', 'solving (O)', 'amp (O)', 'ma...",no_relation
7,7,7,7,4983,2021-10-15 13:49:36+00:00,10,twitter,a7 2/2: investing in social enterprises that h...,a7 2/2: investing in social enterprises that h...,a7 2/2: investing in social enterprises that h...,...,positive,poverty,NaN,"[(0, 116), (0, 0)]",[],"[(0, 9), (0, 20), (0, 27), (0, 39), (0, 48), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['investing (O)', 'enterprise (O)', 'proven (O...",no_relation
8,8,8,8,1239,2019-10-02 08:55:53+00:00,4,twitter,to make the biggest change across africa; youn...,to make the biggest change across africa; youn...,to make the biggest change across africa; youn...,...,positive

In [137]:
# final_df=df.copy()
# final_df=final_df
final_df.shape

(22036, 71)

In [138]:
final_df.head(50)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,...,BERT_nlptown,cleaned_theme,text_in_spans,entity_pairs,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels
0,0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,...,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...",no_relation
1,1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,...,negative,poverty,NaN,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['waste (O)', 'public (O)', 'money (O)', 'worr...",no_relation
2,2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,...,negative,poverty,NaN,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['ca (O)', 'nt (O)', 'truely (O)', 'fixed (O)'...",no_relation
3,3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,...,positive,poverty,NaN,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['made (O)', 'know (O)', 'web (O)', 'medium (O...",no_relation
4,4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,...,negative,poverty,african,"[(0, 18), (0, 18)]",['MISC'],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), 'MISC')]","['do (O)', 'nt (O)', 'worry (O)', 'african (B-...",no_relation
5,5,5,5,1356,2019-08-09 16:40:01+00:00,42,twitter,at this yearâ€™s #baobab2019 #mcfscholars pitc...,at this yearâ€™s #baobab2019 #mcfscholars pitc...,at this yearâ€trade_marks #baobab2019 #mcfscho...,...,positive,poverty,yeartrademarks,"[(0, 14), (0, 14)]",['ORG'],"[(0, 14), (0, 22), (0, 31), (0, 42), (0, 47), ...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(1.5791925191879272, (0, 1), 'ORG')]","['yeartrademarks (B-ORG)', 'pitched (O)', 'cre...",no_relation
6,6,6,6,6008,2021-07-29 08:59:45+00:00,0,twitter,zana africa: solving social &amp; market chall...,zana africa: solving social &amp; market chall...,zana africa: solving social &amp; market chall...,...,positive,poverty,zana kenya,"[(0, 4), (0, 113)]","['ORG', 'LOC']","[(0, 4), (0, 12), (0, 16), (0, 23), (0, 33), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0.8383952379226685, (0, 1), 'ORG'), (2.63775...","['zana (B-ORG)', 'solving (O)', 'amp (O)', 'ma...",no_relation
7,7,7,7,4983,2021-10-15 13:49:36+00:00,10,twitter,a7 2/2: investing in social enterprises that h...,a7 2/2: investing in social enterprises that h...,a7 2/2: investing in social enterprises that h...,...,positive,poverty,NaN,"[(0, 116), (0, 0)]",[],"[(0, 9), (0, 20), (0, 27), (0, 39), (0, 48), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['investing (O)', 'enterprise (O)', 'proven (O...",no_relation
8,8,8,8,1239,2019-10-02 08:55:53+00:00,4,twitter,to make the biggest change across africa; youn...,to make the biggest change across africa; youn...,to make the biggest change across africa; youn...,...,positive

In [139]:
final_df['predicted_labels'].unique()

array(['no_relation', 'per:origin', 'org:country_of_headquarters',
       'per:title', 'org:alternate_names', 'org:top_members/employees',
       'per:countries_of_residence', 'org:founded_by',
       'org:city_of_headquarters', 'per:charges', 'org:website',
       'org:political/religious_affiliation', 'per:parents',
       'per:siblings', 'org:stateorprovince_of_headquarters',
       'per:stateorprovinces_of_residence', 'per:religion',
       'per:cause_of_death', 'per:alternate_names', 'per:date_of_birth',
       'per:date_of_death', 'org:members',
       'org:number_of_employees/members', 'org:founded'], dtype=object)

In [ ]:

# print(seqeval.metrics.classification_report([final_labels], [final_predictions], digits=4)) 

In [140]:
# final_df.columns
df=final_df.copy()
df=df.astype(str)
df.shape

(22036, 71)

In [141]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import pandas as pd

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Assuming 'category' is the column containing labels
# Convert the 'category' column to a string representation

df['category'] = df['categories'].apply(lambda x: str(x))
num_labels = len(df['category'].unique())

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Add label ids to the DataFrame
df['label_id'] = df['category'].astype('category').cat.codes

# # Convert 'head' and 'tail' columns to lists
# df['Head'] = df['Head'].apply(lambda x: [x])
# df['Tail'] = df['Tail'].apply(lambda x: [x])

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, df, max_length=64):
        self.df = df
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        inputs = tokenizer(
            self.df.iloc[idx]["Text_lemma"],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        label = torch.tensor(self.df.iloc[idx]["label_id"]).long()  # Convert to long for BCEWithLogitsLoss
        return {"input_ids": inputs["input_ids"].squeeze(), "attention_mask": inputs["attention_mask"].squeeze(), "label": label}

# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_joint_entity_relation",
    per_device_train_batch_size=8,
    save_total_limit=1,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
)

# Training in chunks
chunk_size = 5000
for start in range(0, len(train_df), chunk_size):
    end = start + chunk_size
    chunk_train_df = train_df.iloc[start:end]

    # Create dataset for the current chunk
    chunk_train_dataset = CustomDataset(chunk_train_df)

    # Update trainer's train_dataset
    trainer.train_dataset = chunk_train_dataset

    # Create evaluation dataset for the current chunk
    chunk_eval_dataset = CustomDataset(test_df)  # You might need to modify this based on your requirements

    # Update trainer's eval_dataset
    trainer.eval_dataset = chunk_eval_dataset

    # Train the model on the current chunk
    trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(chunk_eval_dataset)

# Print evaluation results
print("***** Evaluation Results *****")
for key, value in results.items():
    print(f"{key}: {value}")

# Get predictions on the test set
predictions = trainer.predict(chunk_eval_dataset)

# Convert logits (numpy array) to PyTorch tensor
predicted_labels = torch.argmax(torch.from_numpy(predictions.predictions), dim=1)

# Convert tensor to list
predicted_labels = predicted_labels.tolist()
true_labels = test_df['label_id'].tolist()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\nAccuracy: {accuracy:.4f}")

# Calculate precision, recall, f1 score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
100,2.019000,1.443617
200,1.335500,1.336431
300,1.301700,1.299590
400,1.260100,1.189841
500,1.160600,1.161655
600,1.065400,1.129826
700,1.008500,1.108697
800,0.894700,1.083690
900,0.816800,1.079617
1000,0.851200,0.999905


Checkpoint destination directory ./bert_joint_entity_relation/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Step,Training Loss,Validation Loss
100,1.070400,1.015553
200,1.027800,0.935387
300,0.906300,0.914896
400,0.828000,0.944584
500,0.817800,0.923487
600,0.812500,0.850845
700,0.638500,0.918941
800,0.590200,0.965088
900,0.637400,0.900316
1000,0.571000,0.952888


Step,Training Loss,Validation Loss
100,0.816700,0.795553
200,0.888400,0.771000
300,0.775800,0.836621
400,0.789200,0.744286
500,0.733300,0.812462
600,0.714000,0.770085
700,0.589100,0.772061
800,0.476200,0.891657
900,0.507700,0.834504
1000,0.573300,0.776202


Step,Training Loss,Validation Loss
100,0.791900,0.817488
200,0.831300,0.736724
300,0.713200,0.677537
400,0.533500,0.803959
500,0.471600,0.768331
600,0.377700,0.832167
700,0.406600,0.833175
800,0.301400,0.820710
900,0.260800,0.835577


***** Evaluation Results *****
eval_loss: 0.8312262892723083
eval_runtime: 19.1341
eval_samples_per_second: 230.374
eval_steps_per_second: 28.797
epoch: 3.0

Accuracy: 0.8280
Precision: 0.8238
Recall: 0.8280
F1 Score: 0.8231


/home/hsibitenda/anaconda3/envs/harriet_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hsibitenda/anaconda3/envs/harriet_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [142]:
final_df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0,textid,Date,Views,sourcetype,alltext,Text_emt,Text_emj,...,BERT_nlptown,cleaned_theme,text_in_spans,entity_pairs,categories,entity,original_word_spans,predictions,tokens_with_labels,predicted_labels
0,0,0,0,18965,2021-08-24,576,youtube,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,geography g.12- unit 4 population of ethiopi...,...,neutral,poverty,ethiopia,"[(0, 36), (0, 36)]",['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...",no_relation
1,1,1,1,532,2019-08-22 12:41:52+00:00,0,twitter,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,what a waste of public money .worry about the ...,...,negative,poverty,NaN,"[(0, 104), (0, 0)]",[],"[(0, 5), (0, 12), (0, 18), (0, 24), (0, 32), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['waste (O)', 'public (O)', 'money (O)', 'worr...",no_relation
2,2,2,2,2876,2020-06-05 08:01:33+00:00,0,twitter,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,you can't truely fixed economic; social and in...,...,negative,poverty,NaN,"[(0, 128), (0, 0)]",[],"[(0, 2), (0, 4), (0, 11), (0, 17), (0, 26), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['ca (O)', 'nt (O)', 'truely (O)', 'fixed (O)'...",no_relation
3,3,3,3,2533,2020-09-15 03:04:00+00:00,12,twitter,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,this made my day!! \nyou know the web and soci...,...,positive,poverty,NaN,"[(0, 127), (0, 0)]",[],"[(0, 4), (0, 9), (0, 13), (0, 20), (0, 27), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",[],"['made (O)', 'know (O)', 'web (O)', 'medium (O...",no_relation
4,4,4,4,6413,2021-08-13 16:05:04+00:00,24,twitter,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,@mczuriel @news24 @itchybyte don't worry yours...,...,negative,poverty,african,"[(0, 18), (0, 18)]",['MISC'],"[(0, 2), (0, 4), (0, 10), (0, 18), (0, 24), (0...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(6.357479095458984, (3, 4), 'MISC')]","['do (O)', 'nt (O)', 'worry (O)', 'african (B-...",no_relation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22031,22031,22031,20121,21501,2021-08-24,357,youtube,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,...,negative,poverty,afghanistan,"[(0, 11), (0, 11)]",['LOC'],"[(0, 11), (0, 19), (0, 22), (0, 24), (0, 33), ...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(7.060326099395752, (0, 1), 'LOC')]","['afghanistan (B-LOC)', 'taliban (O)', 'ca (O)...",org:country_of_headquarters
22032,22032,22032,20125,18810,2021-08-24,466,youtube,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,...,negative,poverty,afghanistan,"[(0, 11), (0, 11)]",['LOC'],"[(0, 11), (0, 19), (0, 22), (0, 24), (0, 33), ...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(7.060326099395752, (0, 1), 'LOC')]","['afghanistan (B-LOC)', 'taliban (O)', 'ca (O)...",org:country_of_headquarters
22033,22033,22033,20128,13319,2021-08-24,431,youtube,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,afghanistan: why the taliban can't be defeat...,...,negative,poverty,afghanistan,"[(0, 11), (0, 11)]",['LOC'],"[(0, 11), (0, 19), (0, 22), (0, 24), (0, 33), ...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(7.060326099395752, (0, 1), 'LOC')]","['afghanistan (B-LOC)', 'taliban (O)', 'ca (O)...",org:country_of_headquarters
22034,22034,22034,20130,19270,2021-08-24,468,youtube,afghan

In [98]:
df=final_df[['textid', 'Date', 'Views', 'sourcetype','Text_lemma', 'entity_pairs','predicted_labels']]
df.head()

,textid,Date,Views,sourcetype,Text_lemma,entity_pairs,predicted_labels
0,18965,2021-08-24,576,youtube,geography g unit population ethiopia horn afri...,"[(0, 36), (0, 36)]",no_relation
1,532,2019-08-22 12:41:52+00:00,0,twitter,waste public money worry economy problem anc o...,"[(0, 104), (0, 0)]",no_relation
2,2876,2020-06-05 08:01:33+00:00,0,twitter,cant truely fixed economic infrastructural pro...,"[(0, 128), (0, 0)]",no_relation
3,2533,2020-09-15 03:04:00+00:00,12,twitter,made know web medium always trying start probl...,"[(0, 127), (0, 0)]",no_relation
4,6413,2021-08-13 16:05:04+00:00,24,twitter,dont worry african focus fixing useless countr...,"[(0, 18), (0, 18)]",no_relation


In [6]:
df.shape

(27545, 7)

In [103]:
import ast
import torch
from tqdm import trange
from transformers import LukeTokenizer, LukeForEntityPairClassification
df['Text_lemma'] = df['Text_lemma'].astype(str)
df['predicted_labels'] = df['predicted_labels'].astype(str)
df['entity_pairs'] = df['entity_pairs'].apply(ast.literal_eval)


In [108]:
df=final_df.copy()

In [113]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'textid', 'Date', 'Views',
       'sourcetype', 'alltext', 'Text_emt', 'Text_emj', 'Text_langue',
       'Text_translate', 'Text_langue2', 'Text_lower', 'Text_punct',
       'Text_tags', 'Text_chat', 'Text_spell', 'Text_langue3',
       'Text_translate2', 'Text_token', 'Text_stop', 'Text_stem', 'Text_lemma',
       'berttopic2', 'cluster_id2', 'llama2_labelone10', 'cluster_labelone10',
       'cluster_id', 'llama2_labelone', 'cluster_labelone', 'Unnamed: 0.1',
       'Text_lemma2', 'Bertscore_Labels', 'Final_Labels',
       'Predicted_Categories3', 'polarity', 'subjectivity', 'Textblob',
       'VADER', 'neg', 'neu', 'pos', 'compound', 'BERT', 'Sentiment_Score',
       'Sentiwordnet', 'llama3_sentiment', 'Emotion', 'Emotional_Polarity',
       'Common_Sentiment32', 'Sentiment2', 'Predicted_Labels',
       'Sentiment2_prediction', 'Sentiment2_roberta',
       'Predicted_Labels_roberta', 'Predicted_Sentiment_finetunned',
       'mapped

## Detecting types of entities in a text

Finally, we detect types of entities in a text using the [fine-tuned model](https://huggingface.co/studio-ousia/luke-large-finetuned-open-entity).

In [117]:
!gdown --id 1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
!tar xzf /content/data.tar.gz

# Place test.json on the working directory
# !cp data/OpenEntity/test.json .

/home/hsibitenda/anaconda3/envs/harriet_env/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
From (redirected): https://drive.google.com/uc?id=1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im&confirm=t&uuid=6e48fb05-11b9-48c6-8bc1-f83ac13141d8
To: /scratch/hsibitenda/harriet/entities_code/data.tar.gz
100%|████████████████████████████████████████| 322M/322M [00:07<00:00, 44.9MB/s]
tar (child): /content/data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
cp: cannot stat 'data/OpenEntity/test.json': No such file or directory


In [118]:
# Define the path to the downloaded tar file
tar_path = '/scratch/hsibitenda/harriet/entities_code/data.tar.gz'
extract_path = '/scratch/hsibitenda/harriet/entities_code/'  # Adjust this if you want to extract somewhere specific

# Check if the tar file exists and extract it
if os.path.exists(tar_path):
    # Extract the tar file to the specified path
    !tar -xzf {tar_path} -C {extract_path}
    # After extraction, check if the test.json file exists and move it to the current directory
    json_file_path = os.path.join(extract_path, 'data/OpenEntity/test.json')
    if os.path.exists(json_file_path):
        !cp {json_file_path} .
        print("File copied successfully!")
    else:
        print("test.json does not exist in the extracted directory.")
else:
    print("The tar file does not exist.")


File copied successfully!


In [119]:
def load_examples(dataset_file):
    with open(dataset_file, "r") as f:
        data = json.load(f)

    examples = []
    for item in data:
        examples.append(dict(
            text=item["sent"],
            entity_spans=[(item["start"], item["end"])],
            label=item["labels"]
        ))

    return examples

In [120]:
test_examples = load_examples("test.json")
test_examples

[{'text': 'On late Monday night , 30th Nov 2009 , Bangladesh Police arrested Rajkhowa somewhere near Dhaka .',
  'entity_spans': [(3, 20)],
  'label': ['time']},
 {'text': 'Leo W. Gerard , president of the steelworkers union , said he and several leaders of the AFL-CIO had organized joint events this week with the Sierra Club and the Alliance for Climate Protection .',
  'entity_spans': [(111, 123)],
  'label': ['event']},
 {'text': 'Peace agreements will only bring further losses and push back our cause , " he added , pointing out that Abbas \'s Fatah party also maintains its own armed wing , the loosely affiliated Al - Aqsa Martyrs Brigades .',
  'entity_spans': [(76, 78)],
  'label': ['person']},
 {'text': 'He was re-elected in the 1887 general election and represented Kaiapoi until the 1890 election , when he retired from the lower house .',
  'entity_spans': [(118, 133)],
  'label': ['organization', 'place']},
 {'text': "Barak , who has been unable to cut Sharon 's 20-point lead ,

In [121]:
# Load the model checkpoint
model = LukeForEntityClassification.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")
model.eval()
model.to("cuda")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")



config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-open-entity were not used when initializing LukeForEntityClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [122]:
batch_size = 128

num_predicted = 0
num_gold = 0
num_correct = 0

all_predictions = []
all_labels = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]
    gold_labels = [example["label"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)

    num_gold += sum(len(l) for l in gold_labels)
    for logits, labels in zip(outputs.logits, gold_labels):
        for index, logit in enumerate(logits):
            if logit > 0:
                num_predicted += 1
                predicted_label = model.config.id2label[index]
                if predicted_label in labels:
                    num_correct += 1

precision = num_correct / num_predicted
recall = num_correct / num_gold
f1 = 2 * precision * recall / (precision + recall)

print(f"\n\nprecision: {precision} recall: {recall} f1: {f1}")

100%|██████████| 16/16 [00:04<00:00,  3.35it/s]



precision: 0.7980295566502463 recall: 0.7657563025210085 f1: 0.781559903511123


In [123]:
text = "Beyoncé lives in Los Angeles."
entity_spans = [(0, 7)]  # character-based entity span corresponding to "Beyoncé"

inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
inputs.to("cuda")
outputs = model(**inputs)

predicted_indices = [index for index, logit in enumerate(outputs.logits[0]) if logit > 0]
print("Predicted entity type for Beyoncé:", [model.config.id2label[index] for index in predicted_indices])

entity_spans = [(17, 28)]  # character-based entity span corresponding to "Beyoncé"
inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
inputs.to("cuda")
outputs = model(**inputs)

predicted_indices = [index for index, logit in enumerate(outputs.logits[0]) if logit > 0]
print("Predicted entity type for Los Angeles:", [model.config.id2label[index] for index in predicted_indices])

Predicted entity type for Beyoncé: ['person']
Predicted entity type for Los Angeles: ['location', 'place']


In [146]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'textid', 'Date', 'Views',
       'sourcetype', 'alltext', 'Text_emt', 'Text_emj', 'Text_langue',
       'Text_translate', 'Text_langue2', 'Text_lower', 'Text_punct',
       'Text_tags', 'Text_chat', 'Text_spell', 'Text_langue3',
       'Text_translate2', 'Text_token', 'Text_stop', 'Text_stem', 'Text_lemma',
       'berttopic2', 'cluster_id2', 'llama2_labelone10', 'cluster_labelone10',
       'cluster_id', 'llama2_labelone', 'cluster_labelone', 'Unnamed: 0.1',
       'Text_lemma2', 'Bertscore_Labels', 'Final_Labels',
       'Predicted_Categories3', 'polarity', 'subjectivity', 'Textblob',
       'VADER', 'neg', 'neu', 'pos', 'compound', 'BERT', 'Sentiment_Score',
       'Sentiwordnet', 'llama3_sentiment', 'Emotion', 'Emotional_Polarity',
       'Common_Sentiment32', 'Sentiment2', 'Predicted_Labels',
       'Sentiment2_prediction', 'Sentiment2_roberta',
       'Predicted_Labels_roberta', 'Predicted_Sentiment_finetunned',
       'mapped

In [149]:
df['entity_spans_list']=df[['entity']]

In [150]:
# df.head()

In [151]:
import torch
import pandas as pd
from transformers import LukeTokenizer, LukeForEntitySpanClassification
import spacy

# ... (previous code remains unchanged)

# ... (definition of extract_info function remains unchanged)

# ... (processing DataFrame in chunks and generating final_df remains unchanged)

new_df_rows = []

# Iterate through each row in 'final_df'
for index, row in df.iterrows():
    # Extract the values from the 'entity_spans_list' column
    entity_spans_lists = row['entity_spans_list']

    # Iterate through each list in 'entity_spans_list'
    for pair in entity_spans_lists:
        # If the length of the pair is greater than 2, split into separate pairs
        if len(pair) > 2:
            for i in range(0, len(pair), 2):
                new_pair = pair[i:i + 2]
                new_df_rows.append({
                    'Text_lemma': row['Text_lemma'],
                    'categories' : row['categories'], #label for _, _, label in predictions]
                    'entity' : row['entity'], #= entity_spans
                    'original_word_spans': row['original_word_spans'], # = original_word_spans
                    'predictions':row['predictions'], #= predictions
                    'tokens_with_labels':row['tokens_with_labels'], # = tokens_with_labels

                    'Entity_Spans_List': [tuple(new_pair)],  # Make sure to store it as a list of tuples
                    # Add other columns from 'row' as needed
                })
        else:
            # Add a new row for each pair of tuples
            new_df_rows.append({
                'Text_lemma': row['Text_lemma'],
                'categories' : row['categories'], #label for _, _, label in predictions]
                'entity' : row['entity'], #= entity_spans
                'original_word_spans': row['original_word_spans'], # = original_word_spans
                'predictions':row['predictions'], #= predictions
                'tokens_with_labels':row['tokens_with_labels'], # = tokens_with_labels

                'Entity_Spans_List': [tuple(pair)],  # Make sure to store it as a list of tuples
                # Add other columns from 'row' as needed
            })

# Create a new DataFrame from the list of rows
new_df = pd.DataFrame(new_df_rows)

# Display the new DataFrame
new_df.head()


,Text_lemma,categories,entity,original_word_spans,predictions,tokens_with_labels,Entity_Spans_List
0,geography g unit population ethiopia horn afri...,['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...","[([,)]"
1,geography g unit population ethiopia horn afri...,['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...","[((,)]"
2,geography g unit population ethiopia horn afri...,['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...","[(0,)]"
3,geography g unit population ethiopia horn afri...,['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...","[(,,)]"
4,geography g unit population ethiopia horn afri...,['LOC'],"[(0, 9), (0, 11), (0, 16), (0, 27), (0, 36), (...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(5.652407646179199, (4, 5), 'LOC')]","['geography (O)', 'g (O)', 'unit (O)', 'popula...","[( ,)]"


In [152]:
# import torch
# import pandas as pd
# from transformers import LukeTokenizer, LukeForEntityClassification
# import json

# # Load the model checkpoint
# model = LukeForEntityClassification.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")
# model.eval()
# model.to("cuda")

# # Load the tokenizer
# tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")

# # Function to process each row of the DataFrame
# def process_row(row):
#     text = row['Text_lemma']
#     entity_spans = [(start, end) for start, end in row['Entity_Spans_List']]

#     # Tokenize the text and prepare inputs for the model
#     inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
#     inputs.to("cuda")

#     # Get model outputs
#     outputs = model(**inputs)

#     # Get predicted indices for entities
#     predicted_indices = [index for index, logit in enumerate(outputs.logits[0]) if logit > 0]

#     # Return predicted entity types as a string
#     return ', '.join([model.config.id2label[index] for index in predicted_indices])

# # Apply the function to each row in the DataFrame and store the predictions in a new column
# df['predicted_entities'] = df.apply(process_row, axis=1)

# # Display the DataFrame with predicted entities
# df.head() #[['Text_lemma', 'predicted_entities']])


In [126]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'textid', 'Date', 'Views',
       'sourcetype', 'alltext', 'Text_emt', 'Text_emj', 'Text_langue',
       'Text_translate', 'Text_langue2', 'Text_lower', 'Text_punct',
       'Text_tags', 'Text_chat', 'Text_spell', 'Text_langue3',
       'Text_translate2', 'Text_token', 'Text_stop', 'Text_stem', 'Text_lemma',
       'berttopic2', 'cluster_id2', 'llama2_labelone10', 'cluster_labelone10',
       'cluster_id', 'llama2_labelone', 'cluster_labelone', 'Unnamed: 0.1',
       'Text_lemma2', 'Bertscore_Labels', 'Final_Labels',
       'Predicted_Categories3', 'polarity', 'subjectivity', 'Textblob',
       'VADER', 'neg', 'neu', 'pos', 'compound', 'BERT', 'Sentiment_Score',
       'Sentiwordnet', 'llama3_sentiment', 'Emotion', 'Emotional_Polarity',
       'Common_Sentiment32', 'Sentiment2', 'Predicted_Labels',
       'Sentiment2_prediction', 'Sentiment2_roberta',
       'Predicted_Labels_roberta', 'Predicted_Sentiment_finetunned',
       'mapped

In [127]:
# df['entity_pairs']

0        [(0, 36), (0, 36)]
1        [(0, 104), (0, 0)]
2        [(0, 128), (0, 0)]
3        [(0, 127), (0, 0)]
4        [(0, 18), (0, 18)]
                ...        
22031    [(0, 11), (0, 11)]
22032    [(0, 11), (0, 11)]
22033    [(0, 11), (0, 11)]
22034    [(0, 11), (0, 11)]
22035    [(0, 11), (0, 11)]
Name: entity_pairs, Length: 22036, dtype: object

In [110]:
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# from sklearn.model_selection import train_test_split
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# import torch
# from torch.utils.data import Dataset
# import pandas as pd
# from seqeval.metrics import classification_report as seqeval_classification_report  # Import seqeval's classification_report


# # Assuming 'predicted_labels' is the column containing relation labels
# num_labels = len(df['predicted_labels'].unique())

# # Load BERT tokenizer and model for sequence classification
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels).to('cuda')  # Adjust num_labels and move the model to CUDA

# # Assuming 'predicted_labels' is the column containing relation labels
# df['predicted_labels'] = df['predicted_labels'].apply(lambda x: str(x))
# num_labels = len(df['predicted_labels'].unique())

# # Add label ids to the DataFrame
# df['label_id'] = df['predicted_labels'].astype('category').cat.codes

# # Convert 'entity_pairs' column to lists of tuples
# # df['entity_spans'] = df['entity_spans'].apply(lambda x: eval(x))

# # Split the dataset
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# class CustomDataset(Dataset):
#     def __init__(self, df, max_length=128):
#         self.df = df
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         entity_pairs = self.df.iloc[idx]["entity_spans"]
#         text = self.df.iloc[idx]["Text_lemma"]

#         inputs = tokenizer(
#             text,
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_length,
#             return_tensors="pt",
#         )

#         # Initialize label tensor with no_relation
#         label = torch.tensor(0).long().to('cuda')

#         # Update label tensor based on entity pairs
#         for start, end in entity_pairs:
#             if start != end:
#                 label = torch.tensor(1).long().to('cuda')  # Assuming 1 represents some_relation

#         return {"input_ids": inputs["input_ids"].squeeze(), "attention_mask": inputs["attention_mask"].squeeze(), "label": label}

# # Training arguments
# training_args = TrainingArguments(
#     output_dir="./bert_relation_classification",
#     per_device_train_batch_size=8,
#     save_total_limit=1,
#     num_train_epochs=3,
#     evaluation_strategy="steps",
#     eval_steps=100,
#     logging_steps=100,
# )

# # Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
# )

# # Training in chunks
# chunk_size = 5000
# for start in range(0, len(train_df), chunk_size):
#     end = start + chunk_size
#     chunk_train_df = train_df.iloc[start:end]

#     # Create dataset for the current chunk
#     chunk_train_dataset = CustomDataset(chunk_train_df)

#     # Update trainer's train_dataset
#     trainer.train_dataset = chunk_train_dataset

#     # Create evaluation dataset for the current chunk
#     chunk_eval_dataset = CustomDataset(test_df)  # You might need to modify this based on your requirements

#     # Update trainer's eval_dataset
#     trainer.eval_dataset = chunk_eval_dataset

#     # Train the model on the current chunk
#     trainer.train()

# # Evaluate the model on the test set
# results = trainer.evaluate(chunk_eval_dataset)

# # Print evaluation results
# print("***** Evaluation Results *****")
# for key, value in results.items():
#     print(f"{key}: {value}")

# # Get predictions on the test set
# predictions = trainer.predict(chunk_eval_dataset)

# # Flatten the predicted_labels
# predicted_labels = [model.config.id2label[index.item()] for index in predictions.predictions.argmax(-1).flatten()]

# # Convert tensor to list for true labels
# true_labels = test_df['predicted_labels'].tolist()

# # Calculate accuracy
# accuracy = accuracy_score(true_labels, predicted_labels)
# print(f"\nAccuracy: {accuracy:.4f}")

# # Calculate precision, recall, f1 score
# precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")


In [112]:

# # Generate classification report using seqeval
# class_report = seqeval_classification_report([true_labels], [predicted_labels], digits=4)
# print("\nClassification Report:\n", class_report)